## Setup

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
data_path = Path('/Users/rancze/Documents/Data/Mismatch_experiments/Cohort0_GCaMP_example/2024-08-08T10-05-26_B3M3')
photometry_path = Path('/Users/rancze/Documents/Data/Mismatch_experiments/Cohort0_GCaMP_example/photometry/B3M3_MMclosedOpen_day2/2024_08_08-12_08_29')

In [ ]:
SessionSettings = utils.read_SessionSettings(data_path, print_contents=True)

## Videography and SLEAP

In [ ]:
# lp.create_flipped_videos(data_path, what_to_flip='VideoData1')

### Run SLEAP inference now

```
In the terminal:
>> conda activate sleap
>> sleap-label

Open "working_project.slp" in "SLEAP_models" directory on the NAS.
SLEAP documentation: https://sleap.ai/tutorials/tutorial.html
```

### Load videography data

In [ ]:
VideoData1, VideoData2, VideoData1_Has_Sleap, VideoData2_Has_Sleap = lp.load_videography_data(data_path)

### SLEAP Quality Control

In [ ]:
VideoData2

In [ ]:
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
VideoData2[VideoData2[columns_of_interest].isnull().all(1)]

In [ ]:
VideoData2[VideoData2[columns_of_interest].isnull().all(1)].index.values

In [ ]:
all_nan_df = VideoData2[VideoData2[columns_of_interest].isnull().all(1)]
all_nan_index_array = all_nan_df.index.values

i=1
for group in lp.find_sequential_groups(all_nan_index_array):
    print(f'NaN frame group {i} with {len(group)} elements')
    i += 1

In [ ]:
# for col in columns_of_interest:
# indices_outside_common_nan_list
# VideoData2[VideoData2[col].isna()].index.values

In [ ]:
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
coordinates_dict=lp.get_coordinates_dict(VideoData2, columns_of_interest)

for col in columns_of_interest:
    plt.figure(figsize=(18,4))
    plt.title(col, fontsize=16)
    plt.plot(VideoData2[col].values)
    plt.show()

In [ ]:
columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

fig, ax = plt.subplots(nrows=11, ncols=1, figsize=(4,44))

for col in columns_of_interest:
    ax[columns_of_interest.index(col)].set_title(col)
    ax[columns_of_interest.index(col)].scatter(coordinates_dict[f'{col}.x'], coordinates_dict[f'{col}.y'])
plt.show()

### SLEAP processing

In [ ]:
VideoData2 = VideoData2.interpolate()

columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
coordinates_dict=lp.get_coordinates_dict(VideoData2, columns_of_interest)

theta = lp.find_horizontal_axis_angle(VideoData2, 'left', 'center')
center_point = lp.get_left_right_center_point(coordinates_dict)

columns_of_interest = ['left', 'right', 'center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
remformatted_coordinates_dict = lp.get_reformatted_coordinates_dict(coordinates_dict, columns_of_interest)
centered_coordinates_dict = lp.get_centered_coordinates_dict(remformatted_coordinates_dict, center_point)
rotated_coordinates_dict = lp.get_rotated_coordinates_dict(centered_coordinates_dict, theta)

columns_of_interest = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
ellipse_parameters_data, ellipse_center_points_data = lp.get_fitted_ellipse_parameters(rotated_coordinates_dict, columns_of_interest)

average_diameter = np.mean([ellipse_parameters_data[:,0], ellipse_parameters_data[:,1]], axis=0)

SleapVideoData2 = process.convert_arrays_to_dataframe(['Seconds', 'Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y'], [VideoData2['Seconds'].values, average_diameter, ellipse_parameters_data[:,2], ellipse_center_points_data[:,0], ellipse_center_points_data[:,1]])

## Loading and Synchronisation 

In [ ]:
conversions = process.calculate_conversions_second_approach(data_path, photometry_path, verbose=False)
# After hardware ONIX clock implementation - this will have to be adapted
# Only photometry will need to be converted

In [ ]:
streams = utils.load_registers(data_path)

Photometry = utils.read_fluorescence(photometry_path)
Photometry['HARP Timestamps'] = conversions['photometry_to_harp_time'](Photometry['TimeStamp'])

OnixAnalogClock = utils.read_OnixAnalogClock(data_path)
OnixAnalogData = utils.read_OnixAnalogData(data_path, binarise=True)
ExperimentEvents = utils.read_ExperimentEvents(data_path) 

photodiode_series = pd.Series(OnixAnalogData[:,0], index=conversions['onix_to_harp_timestamp'](OnixAnalogClock))

In [ ]:
# Adding Photometry, Eye Movements and Photodiode to the streams
streams = process.reformat_and_add_many_streams(streams, Photometry, 'Photometry', ['CH1-410', 'CH1-470', 'CH1-560'], index_column_name='HARP Timestamps')
#streams = process.reformat_and_add_many_streams(streams, SleapVideoData2, 'SleapVideoData2', ['Ellipse.Diameter', 'Ellipse.Angle', 'Ellipse.Center.X', 'Ellipse.Center.Y'])
streams = process.add_stream(streams, 'ONIX', photodiode_series, 'Photodiode')

In [ ]:
#this is very memory hungry and kills the kernel even with 20ms resampling period (even though the initial value in the notebook was 0.1ms)
#on Ede's mac, it works with down to 25ms but takes a very long time 
#need to look into pad_and_resample, seems to be using U64 for example 
#also, there is a lot of data left in the memory after this step, is this necess
_ = process.get_timepoint_info(streams, print_all=True)
resampled_streams = process.pad_and_resample(streams, resampling_period='25ms', method='linear')
_ = process.get_timepoint_info(resampled_streams, print_all=True)

In [ ]:
# Applying linear and angular conversion to Optical tracking sensor streams
# OpticalTrackingRead0X(46) converted to centimeters per second
# OpticalTrackingRead0Y(46) covnerted to degrees per second
resampled_streams['H1']['OpticalTrackingRead0X(46)'] = process.running_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0X(46)']*100)
resampled_streams['H1']['OpticalTrackingRead0Y(46)'] = process.rotation_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0Y(46)'])

## Usage

In [ ]:
running = resampled_streams['H1']['OpticalTrackingRead0X(46)']
rotation = resampled_streams['H1']['OpticalTrackingRead0Y(46)']
photometry = resampled_streams['Photometry']['CH1-470']
# eye_movements = resampled_streams['SleapVideoData2']['Ellipse.Center.X']

In [ ]:
t = (running.index - utils.harp.REFERENCE_EPOCH).total_seconds() #date stamp to seconds 

A = t[0] #time window beginning in harp seconds, min different from dataset to dataset, None means from start
B = t[-1] #time window end in harp seconds, None means to end 
# A = 444100 
# B = A + 50

photodiode_x, photodiode_y = process.select_from_photodiode_data(OnixAnalogClock, OnixAnalogData, A, B, conversions)

fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(12,24))

ax[0].plot(t, running)
ax[0].set_title('Running')
ax[0].set_xlim([A,B])
ax[0].set_xlabel('time (seconds)')
ax[0].set_ylabel('running speed (cm/s)')

ax[1].plot(t, photometry)
ax[1].set_title('CH1-470')
ax[1].set_xlim([A,B])
ax[1].set_xlabel('time (seconds)')
ax[1].set_ylabel('signal magnitude')

# ax[2].plot(t, eye_movements)
# ax[2].set_title('Eye Movements')
# ax[2].set_xlim([A,B])
# ax[2].set_xlabel('time (seconds)')
# ax[2].set_ylabel('horizontal eye position (pixels)')

ax[3].plot(process.convert_datetime_to_seconds(photodiode_x), photodiode_y[:,0])
ax[3].set_title('Photodiode')
ax[3].set_xlim([A,B])
ax[3].set_xlabel('time (seconds)')
ax[3].set_ylabel('photodiode signal')

plt.show()

In [ ]:
streams_to_save_pattern = {'H1': ['OpticalTrackingRead0X(46)', 'OpticalTrackingRead0Y(46)'], 'H2': ['Encoder(38)'], 'Photometry': ['CH1-410', 'CH1-470', 'CH1-560'], 'SleapVideoData1': ['Ellipse.Diameter', 'Ellipse.Center.X', 'Ellipse.Center.Y'], 'SleapVideoData2': ['Ellipse.Diameter', 'Ellipse.Center.X', 'Ellipse.Center.Y'], 'ONIX': ['Photodiode']}
streams_to_save_pattern

In [ ]:
process.save_streams_as_h5(data_path, resampled_streams, streams_to_save_pattern)